In [1]:
from dotenv import load_dotenv
load_dotenv() # take environment variables from .env.

True

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

model = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # api_key="...",  # if you prefer to pass api key in directly instaed of using env vars
    # base_url="...",
    # organization="...",
    # other params...
)
prompt = PromptTemplate(
        template="Explain about {topic} in 100 words.",
        input_variables=["topic"]
    )

chain_without_parser = prompt | model
result_without = chain_without_parser.invoke({"topic": "Large Language Models"})
print(result_without)

content="Large Language Models (LLMs) are advanced artificial intelligence systems designed to understand, generate, and manipulate human language. They are built using deep learning techniques, particularly neural networks with billions of parameters, enabling them to process vast amounts of text data. LLMs, like OpenAI's GPT series, are trained on diverse datasets, allowing them to perform a wide range of language tasks, including translation, summarization, and conversation. Their ability to generate coherent and contextually relevant text makes them valuable for applications in customer service, content creation, and more. However, they also raise ethical concerns regarding bias, misinformation, and privacy." additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 126, 'prompt_tokens': 17, 'total_tokens': 143, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0},

In [4]:
parser = StrOutputParser()
chain_with_parser = prompt | model | parser
result_with = chain_with_parser.invoke({"topic": "Large Language Models"})
print(result_with)

Large Language Models (LLMs) are advanced artificial intelligence systems designed to understand, generate, and manipulate human language. They are built using deep learning techniques, particularly neural networks with billions of parameters, trained on vast datasets comprising text from the internet, books, and other sources. LLMs, like OpenAI's GPT series, excel in tasks such as translation, summarization, and conversation by predicting the next word in a sentence. Their ability to generate coherent and contextually relevant text makes them valuable for applications in customer service, content creation, and more, although they also raise ethical concerns regarding bias and misinformation.


In [5]:
prompt_without_parser = PromptTemplate(
        template="Analyze the movie {movie_title} and provide details about genre, rating, and key themes.",
        input_variables=["movie_title"]
    )

In [7]:
chain_without_parser = prompt_without_parser | model
result_without = chain_without_parser.invoke({"movie_title": "The Matrix"})
print(result_without)

content='"The Matrix," released in 1999 and directed by the Wachowskis, is a groundbreaking film that blends several genres, including science fiction, action, and cyberpunk. It is renowned for its innovative special effects, particularly the use of "bullet time," and its philosophical depth.\n\n### Genre:\n- **Science Fiction**: The film explores futuristic concepts such as artificial intelligence, virtual reality, and dystopian futures.\n- **Action**: Known for its intense and stylized action sequences, including martial arts and gunfights.\n- **Cyberpunk**: Features a high-tech, low-life aesthetic typical of the cyberpunk genre, with themes of rebellion against oppressive systems.\n\n### Rating:\n- **MPAA Rating**: R (Restricted) for sci-fi violence and brief language.\n- **Critical Reception**: "The Matrix" received widespread critical acclaim for its innovative visual effects, action sequences, and thought-provoking narrative. It holds a high rating on review aggregator sites like

In [8]:
from langchain_core.output_parsers import JsonOutputParser

In [9]:
parser = JsonOutputParser()
format_instructions = parser.get_format_instructions()


prompt_with_parser = PromptTemplate(
    template="""Analyze the movie {movie_title} and provide details about genre, rating, and key themes.
    {format_instructions}""",
    input_variables=["movie_title"],
    partial_variables={"format_instructions": format_instructions}
    )
    
chain_with_parser = prompt_with_parser | model | parser
result_with = chain_with_parser.invoke({"movie_title": "The Matrix"})
print(result_with)

{'title': 'The Matrix', 'genre': ['Science Fiction', 'Action', 'Adventure'], 'rating': 'R', 'key_themes': ['Reality vs. Illusion', 'Control and Freedom', 'Technology and Humanity', 'Identity and Self-discovery', 'Rebellion and Resistance']}


In [10]:
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List

In [11]:
# Define Pydantic model
class Person(BaseModel):
    name: str = Field(description="Person's full name")
    age: int = Field(description="Person's age in years", ge=0, le=150)
    occupation: str = Field(description="Person's job or profession")
    skills: List[str] = Field(description="List of person's key skills")

In [12]:
prompt_without_parser = PromptTemplate(
    template="Create a fictional person profile for a {profession}.",
    input_variables=["profession"]
)

chain_without_parser = prompt_without_parser | model
result_without = chain_without_parser.invoke({"profession": "data scientist"})

print("BEFORE PARSER:")
print("Prompt used:", prompt_without_parser.invoke({"profession": "data scientist"}))
print(f"Type: {type(result_without)}")
print(f"result_without: {result_without}")
print()

# WITH PARSER - Modified prompt with format instructions
parser = PydanticOutputParser(pydantic_object=Person)
format_instructions = parser.get_format_instructions()


prompt_with_parser = PromptTemplate(
    template="Create a fictional person profile for a {profession}.\n{format_instructions}",
    input_variables=["profession"],
    partial_variables={"format_instructions": format_instructions}
)

chain_with_parser = prompt_with_parser | model | parser
result_with = chain_with_parser.invoke({"profession": "data scientist"})

print("AFTER PARSER:")
print("Prompt used:", prompt_with_parser.invoke({"profession": "data scientist"}))
print(f"Type: {type(result_with)}")
print(f"result_with: {result_with}")

BEFORE PARSER:
Prompt used: text='Create a fictional person profile for a data scientist.'
Type: <class 'langchain_core.messages.ai.AIMessage'>
result_without: content='**Name:** Dr. Alex Chen\n\n**Age:** 34\n\n**Location:** San Francisco, California, USA\n\n**Education:**\n- Ph.D. in Computer Science, specializing in Machine Learning, Stanford University\n- M.S. in Statistics, University of California, Berkeley\n- B.S. in Mathematics and Computer Science, University of Illinois at Urbana-Champaign\n\n**Professional Experience:**\n\n1. **Senior Data Scientist at TechNova Solutions (2020 - Present)**\n   - Lead a team of data scientists in developing predictive models for client projects across various industries, including healthcare, finance, and e-commerce.\n   - Implemented a machine learning algorithm that increased client sales forecasting accuracy by 25%.\n   - Spearheaded the development of an internal tool that automates data cleaning processes, reducing project setup time by 4